# Criando o jogo do dinossauro

Vamos inicialmente utilizar o conceito de _sprites_ que nada mais é do que os quadros que irão animar o personagem do jogo. Tais quadros iremos chamá-los na classe que designará cada elemento que irá aparecer na tela. Começaremos com a classe do herói (o dinossauro ou qualquer boneco que faça o papel dele no jogo).

In [10]:
# Antes de qualquer coisa temos que fazer o import da biblioteca
import pygame
from pygame.locals import *

# Dimensões da tela
SIZE = WIDTH, HEIGHT = 800, 640

Para fazermos personagens animados num jogo teremos sempre que criar uma classe com os atributos desse personagem e ela sempre será uma classe herdeira de ``pygame.sprite.Sprites``, esse é o motivo para que esse objeto apareça entre parêntesis ao definir a classe de um elemento do nosso jogo.

In [11]:
class Boneco(pygame.sprite.Sprite):
    # Essas duas linhas sempre serão usadas ao definir uma classe para o pygame
    def __init__(self):
        super(Boneco, self).__init__()

        # o atributo images dessa classe contém uma lista com todas as imagens
        # sempre abrimos elas com o método load
        self.images = [pygame.image.load(f'images/walk{i}.png') for i in range(1,11)]
        # a linha acima pode ser substituída por
        self.images = [] # iniciar uma lista vazia
        self.images.append(pygame.image.load(f'images/walk1.png')) # adiciona a primeira imagem a lista
        self.images.append(pygame.image.load(f'images/walk2.png')) # a segunda
        self.images.append(pygame.image.load(f'images/walk3.png')) # ...
        self.images.append(pygame.image.load(f'images/walk4.png'))
        self.images.append(pygame.image.load(f'images/walk5.png'))
        self.images.append(pygame.image.load(f'images/walk6.png'))
        self.images.append(pygame.image.load(f'images/walk7.png'))
        self.images.append(pygame.image.load(f'images/walk8.png'))
        self.images.append(pygame.image.load(f'images/walk9.png'))
        self.images.append(pygame.image.load(f'images/walk10.png')) # finalmente a décima
        # escolha o método que achar mais adequado

        self.index = 0 # atributo de índice 
        self.image = self.images[self.index] # imagem atual que será desenhada na tela
        # para o retângulo que define onde o personagem está na tela  fornecemos quatro parâmteros
        # posição x, posição y, largura do rtângulo e altura do retângulo
        self.rect = pygame.Rect(5,112,150,198) # retângulo que irá compor o herói
        # brinque um pouco principalmente mudando a posição x e y do personagem, y em específico define
        # a posição do chão`

        self.pulo = False # atributo útil para compor o pulo do personagem


    # Esse método da classe permite executar o pulo do personagem
    def jump(self):
        # Esse método será acessado no jogo quando a tecla for pressionada
        self.pulo = True # o atributo de pulo agora é verdadeiro, se ele está no pulo
        self.t = 0 # atributo útil para dinâmica do salto


    # Esse método
    def update(self):

        # essa primeira parte é genérica para qualquer objeto que será animado
        self.index += 1 # avança pra próxima imagem
        self.index %= 10 # mantém o índice sempre dentro do limite de 0 a 9 (dez imagens)
        self.image = self.images[self.index] # seleciona a imagem que será exibida de acordo ao valor do índice
        
        # essa condicional ocorre quando há o acionamento do pulo
        if self.pulo == True:
            # dinâmica do salto
            v0 = -25 # velocidade inicial negativa, sobe
            g = 2 # gravidade positiva, puxa pra baixo
            t = self.t # o tempo é acionado enquanto a dinâmica está ativa
            self.rect.y += v0+g*t # a altura é incrementada de acordo a dinâmica de queda-livre
            self.t += 1 # o tempo é incrementado em uma unidade
            # o chão deve estar definido por um expressão condicional
            if self.rect.y > 112: # if que define o chão
                self.rect.y = 112 # o y do boneco tem que tá no mesmo valor quando ele foi definido
                self.pulo = False # a dinâmica de salto é desligada
                self.t = 0 # o cronômetro é zerado para aguardar o próximo salto

O _sprite_ que irá compor o chão usará uma técnica de animação um pouco diferente que é chamada de folha de _sprites_, isto é, uma imagem contendo todos os quadros de animação é carregada e as imagens são selecionadas a partir de subregiões daquela imagem. A classe possui uma estrutura básica semelhante a classe acima, só não terá uma dinâmica de movimento ao acinarmos alguma tecla de controle do jogo, afinal o chão se move como parte da dinâmica do jogo.

In [12]:
# a imagem deve ser carregada globalmente 
folhadeanimação = pygame.image.load('images/offline-sprite-2x-black.png')

class Chao(pygame.sprite.Sprite):
    # método de início da classe habitual
    def __init__(self, x, xf) -> None: # aqui usamos parâmetros para passarmos ao criarmos os objetos
        # x é a posição do chão na tela e xf é a posição da imagem que queremos pegar
        # a imagem é bem grande e oferece muitas possibilidades de posição da imagem para variar o terreno
        super(Chao, self).__init__()

        # selecionar a subimagem da folha de sprites
        # xf é a posição x em pixels para pegarmos a subimagem
        # 102 é a coordenada y em pixel onde o chão está localizado, abra a imagem para conferir
        # 803 é a largura da imagem selecionada
        # 28 é a altura da seleção, se y=102, resta 28 pixels de altura pois a imagem toda possui
        # 130 pixels de altura
        self.image = folhadeanimação.subsurface((xf,102),(803,28))
        # usa esse método get_rect() da imagem para não precisar se preocupar com os parâmetros do retângulo
        self.rect = self.image.get_rect()
        # talvez o chão tenha que ser ajustado para que o boneco pareça caminhar melhor, então ajuste para
        # algo perto de 112
        self.rect.y = 112 
        self.rect.x = x # onde o chão será mostrado na tela, a partir de x

    def update(self) -> None:
        # o update apenas necessita de duas linhas:
        self.rect.move_ip((-10,0)) # mover o retângulo do chão da direita pra esquerda
        # essa próxima linha define a condição de contorno para que o chão percorra duas vezes o tamanho da tela
        # isso garante uma transição mais suave quando o chão começa a desaparecer e reaparecer no lado oposto
        self.rect.x = (self.rect.x+WIDTH)%(2*WIDTH)-WIDTH

Definidas as classes, vamos iniciar o jogo, criar os elementos que serão mostrados na tela e colocá-los dentro de um grupo do _pygame_ que irá garantir que eles sejam desenhados na tela ao longo do _looping_ principal.

In [14]:
# O início antes do loop é sempre idêntico, iniciar e selecionar o tamanho da tela, criar o relógio, etc.
pygame.init()

tela = pygame.display.set_mode(SIZE)
relógio = pygame.time.Clock()
rodando = True

# Agora vamos criar os objetos que vão aparecer na tela
heroi = Boneco()
piso1 = Chao(0,0)
piso2 = Chao(800,1001) # a imagem tem mais de 2000 pixels de largura, pegar um pedaço que começa em 1001 é tranquilo

# agora agrupamos os elementos criados
grupo = pygame.sprite.Group() # cria o grupo
grupo.add(heroi) # adiciona o boneco
grupo.add(piso1) # adiciona o primeiro piso
grupo.add(piso2) # adiciona o segundo piso

# Finalmente o looping principal do jogo
while rodando:
    # Inspetor de eventos
    for evento in pygame.event.get():
        # Evento básico de sair ao pressionar o botão de sair da janela
        if evento.type == QUIT:
            rodando = False
        # Se uma tecla do teclado foi pressionada
        if evento.type == KEYDOWN:
            # Se essa tecla foi o espaço
            if evento.key == K_SPACE:
                heroi.jump() # Executa o pulo
    
    # Agora façamos apenas desenhos
    tela.fill((0,0,0)) # pinta a tela de preto
    grupo.update() # Atualiza os elementos que vão aparecer na tela
    grupo.draw(tela) # desenha todos os elementos na tela
    pygame.display.flip() # Desenha os elementos que mudaram na tela

    relógio.tick(10) # Ajusta o looping para 10 quadros por segundo

# Game Over
pygame.quit()


Alinhe o chão com o pé do boneco que anda.